In [2]:
from itertools import combinations
from z3 import *
import math
import numpy as np
import time as t

In [41]:
def toBinary(num, length = None, dtype = int):
    num_bin = bin(num).split("b")[-1]
    if length:
        num_bin = "0"*(length - len(num_bin)) + num_bin
    num_bin = [dtype(int(s)) for s in num_bin]
    return num_bin


# def toInteger(bool_list):
#     new_list = []
#     if type(bool_list[0]) == BoolRef:
#         for b in bool_list:
#             try:
#                 new_list.append(bool(b))
#             except Z3Exception:
#                 new_list.append(False)
#     else:
#         new_list = bool_list

#     binary_string = ''.join('1' if b else '0' for b in new_list)
#     return int(binary_string, 2)
def toInteger(bool_list):

    binary_string = ''.join('1' if b else '0' for b in bool_list)

    return int(binary_string, 2)

In [40]:
b = Bool('rho_0')
bool(If(b, 1, 0))

Z3Exception: Symbolic expressions cannot be cast to concrete Boolean values.

In [4]:
def at_least_one(bool_vars):
    return Or(bool_vars)

def at_most_one(bool_vars, name):
    constraints = []
    n = len(bool_vars)
    s = [Bool(f"s_{name}_{i}") for i in range(n - 1)]
    constraints.append(Or(Not(bool_vars[0]), s[0]))
    constraints.append(Or(Not(bool_vars[n-1]), Not(s[n-2])))
    for i in range(1, n - 1):
        constraints.append(Or(Not(bool_vars[i]), s[i]))
        constraints.append(Or(Not(bool_vars[i]), Not(s[i-1])))
        constraints.append(Or(Not(s[i-1]), s[i]))
    return And(constraints)

def exactly_one(bool_vars, name):
    return And(at_least_one(bool_vars), at_most_one(bool_vars, name))

def lesseq(a, b):
  """
  a and b are lists of variables of equal length containing each the binary
  encoding of an integer number. The constraint is true if a is less than or
  equal to b
  """
  constraints = []
  constraints.append(Or(Not(a[0]),b[0]))
  for i in range(1,len(a)):
    constraints.append(Implies(And([a[k] == b[k] for k in range(i)]), Or(Not(a[i]),b[i])))
  return And(constraints)

def equals(a, b):
  return And([a[i] == b[i] for i in range(len(a))])

In [5]:
def one_bit_full_adder(a, b, c_in, s, c_out):
    xor_ab = Xor(a, b)
    constr_1 = s == Xor(xor_ab, c_in)
    constr_2 = c_out == Or(And(xor_ab, c_in), And(a, b))
    return And(constr_1, constr_2)

def full_adder(a, b, d, name= ""):
  # d has to have proper length, which is not being checked for now
  if len(a)==len(b):
    n = len(a)
  elif  (len(a)>len(b)):
    n = len(a)
    b = [BoolVal(False)] * (len(a) - len(b)) + b  
  else:
    n = len(b)
    a = [BoolVal(False)] * (len(b) - len(a)) + a

  c = [Bool(f"carry_{name}_{i}") for i in range(n)] + [BoolVal(False)]

  constr = []

  for i in range(n):
    constr.append(one_bit_full_adder(a= a[n-i-1], b= b[n-i-1], c_in= c[n - i], s= d[n - i - 1], c_out= c[n - i - 1]))
  constr.append(Not(c[0]))   # this constraint checks whether d has indeed proper length. If a+b overflows, the last carry_out has to be 1. So I assert it is 0
  return And(constr)

def sum_vec(vec, s, name= ""):
  # aggiungere la possibilità che vec e s abbiano dimensione diversa
  if len(vec) == 2:
    return full_adder(a= vec[0], b= vec[1], d= s, name= name)
  elif len(vec) == 1:
    return equals(vec[0], s)
  else:
    partial = [[Bool(f"p_{name}_{i}_{b}") for b in range(len(vec[0]))] for i in range(len(vec) - 2)]
    constr = []
    constr.append(full_adder(vec[0], vec[1], partial[0], name= name+"0"))
    for i in range(len(vec)-3):
      constr.append(full_adder(partial[i], vec[i+2], partial[i+1], name= name+str(i+1)))
    constr.append(full_adder(partial[-1], vec[-1], s, name= name+"last"))
    return And(constr)

def maximum(vec, maxi, name= ""):
  if len(vec) == 1:
    return equals(vec[0], maxi)
  elif len(vec) == 2:
    constr1 = Implies(lesseq(vec[0], vec[1]), equals(vec[1], maxi))
    constr2 = Implies(Not(lesseq(vec[0], vec[1])), equals(vec[0], maxi))
    return And(constr1, constr2)
  
  par = [[Bool(f"maxpar_{name}_{i}_{b}") for b in range(len(maxi))] for i in range(len(vec)-2)]
  constr = []

  constr.append(Implies(lesseq(vec[0], vec[1]), equals(vec[1], par[0])))
  constr.append(Implies(Not(lesseq(vec[0], vec[1])), equals(vec[0], par[0])))

  for i in range(1, len(vec)-2):
    constr.append(Implies(lesseq(vec[i+1], par[i-1]), equals(par[i-1], par[i])))
    constr.append(Implies(Not(lesseq(vec[i+1], par[i-1])), equals(vec[i+1], par[i])))

  constr.append(Implies(lesseq(vec[-1], par[-1]), equals(par[-1], maxi)))
  constr.append(Implies(Not(lesseq(vec[-1], par[-1])), equals(vec[-1], maxi)))
  
  return And(constr)

In [45]:
class Courier:
    def __init__(self, m, n, l, s, D):
        self.m = m
        self.n = n
        self.l = l
        self.s = s
        self.D = np.array(D)
        self.correspondences = None
        self.corr_inverse = None
        
    def preprocess_instance(self):
        self.correspondences = np.argsort(self.l)[::-1]
        transformation_function = sorted(zip(self.correspondences, np.arange(self.m)))
        self.corr_inverse = np.fromiter((x for _, x in transformation_function), dtype= int)
        
        self.l = sorted(self.l, reverse= True)
        # np.array(inst.l)[corr_inverse] to invert the sorting of l

In [7]:
m = 3
n = 7
l = [15, 10, 10]
s = [3, 2, 6, 8, 5, 4, 4]
D = [[0, 3, 3, 6, 5, 6, 6, 2],
     [3, 0, 4, 3, 4, 7, 7, 3],
     [3, 4, 0, 7, 6, 3, 5, 3],
     [6, 3, 7, 0, 3, 6, 6, 4],
     [5, 4, 6, 3, 0, 3, 3, 3],
     [6, 7, 3, 6, 3, 0, 2, 4],
     [6, 7, 5, 6, 3, 2, 0, 4],
     [2, 3, 3, 4, 3, 4, 4, 0]]

instance = Courier(m,n,l,s,D)

In [8]:
def load_instance(num: int):
  if num < 10:
    num = "0"+str(num)
  else:
    num = str(num)
  
  file = open(f"../../input/inst{num}.dat", 'r')
  
  m = int(file.readline())
  n = int(file.readline())
  l = [int(x) for x in file.readline().split(" ") if x!= ""]
  s = [int(x) for x in file.readline().split(" ") if x!= ""]
  D = []
  for i in range(n+1):
      D.append([int(x) for x in file.readline().split(" ") if x!= "\n" if x!= ""])
  
  instance = Courier(m, n, l, s, D)
  instance.preprocess_instance()
  return instance


In [9]:
inst = load_instance(2)
print(inst.l)
print(inst.corr_inverse)



[195, 190, 190, 185, 185, 185]
[2 5 4 1 0 3]


In [78]:
%%time
def mtps(instance):
  start_time = t.time()
  m = instance.m
  n = instance.n
  # l = instance.l
  # s = instance.s
  # D = instance.D

  maxs = np.max(instance.s)
  maxl = np.max(instance.l)
  maxD = np.max(instance.D)
  maxDBin = int(np.ceil(np.log2(maxD)))
  maxDist = n * maxD
  maxDistBin = int(np.ceil(np.log2(maxDist)))
  maxWeight = np.sum(instance.s)
  maxWeightBin = int(np.ceil(np.log2(maxWeight)))
  maxDBin = maxDistBin

  sol = Solver()

  X = [[[Bool(f"x_{i}_{j}_{k}") for k in range(0,n+1)] for j in range(n)] for i in range(m)]

# save the values of the instance
  l = [[BoolVal(b) for b in toBinary(instance.l[i], length= maxWeightBin)] for i in range(m)]
  s = [[BoolVal(b) for b in toBinary(instance.s[j], length= maxWeightBin)] for j in range(n)]
  D = [[[BoolVal(b) for b in toBinary(instance.D[i][j], length= maxDBin)] for j in range(n+1)] for i in range(n+1)]

# each cell has only one value.
  for i in range(m):
    for j in range(n):
      sol.add(exactly_one(X[i][j],f"valid_cell_{i}_{j}"))

# each element except zero should be seen only once inside the matrix
  for k in range(1,n+1):
      sol.add(exactly_one([X[i][j][k] for i in range(m) for j in range(n)],f"valid_k{k}"))

# ordering constraint: if the courier does not take the kth pack at position j, also at position j+1 doesnt
  for i in range(m):
    for j in range(n-1):
      sol.add(Implies(X[i][j][0], X[i][j+1][0]))
# each courier can only deliver items whose total size doesn't exceed limit
  """
  W_par is a matrix that contains for each courier i= 0..m-1, the binary 
  encoding of the weight of the item k= 0..n ONLY IF it is carried by i,
  0 otherwise
  The weight of each item, or 0 is summed and saved into the matrix W_tot
  """
  W_par = [[[Bool(f"partial_weight_{i}_{k}_{b}") for b in range(maxWeightBin)] for k in range(n)] for i in range(m)]
  W_tot = [[Bool(f"total_weight_{i}_{b}") for b in range(maxWeightBin)] for i in range(m)]

  for i in range(m):
    # 1. copy the weight from s to partial weight if needed
    for k in range(n):
      sol.add( Implies( at_least_one([X[i][j][k+1] for j in range(n)]), And([W_par[i][k][b] == s[k][b] for b in range(maxWeightBin)])))
      sol.add( Implies( Not(at_least_one([X[i][j][k+1] for j in range(n)])), And([Not(W_par[i][k][b]) for b in range(maxWeightBin)])))
    # 2. compute the sum of the partial weights for each courier
    sol.add(sum_vec(W_par[i], W_tot[i], name= f"weight_{i}"))
    # 3. for each courier the sum should be less than or equal to the max load size
    sol.add(lesseq(W_tot[i], l[i]))

  D_par = [[[Bool(f"partial_distances_{i}_{j}_{b}") for b in range(maxDBin)] for j in range(n+1)] for i in range(m)]
  D_tot = [[ Bool(f"total_distances_{i}_{b}") for b in range(maxDistBin)] for i in range(m)]
  
  # obj = [Bool(f"obj_{b}") for b in range(maxDistBin)]

  rho = [Bool(f"rho_{b}") for b in range(maxDistBin)]

  # 1. copy the distances from D to partial distances if needed
  for i in range(m):
    # from deposit to first place
    ## sol.add(Implies(X[i][0][0], equals(D_par[i][0], D[n][n])))
    sol.add(Implies(X[i][0][0], And([Not(D_par[i][j][b]) for j in range(n+1) for b in range(maxDBin)])))
    
    for k in range(1, n+1):
      sol.add(Implies(Not(X[i][0][0]), Implies(X[i][0][k], equals(D_par[i][0], D[n][k-1]))))
      
    # from j - 1 to j
    for j in range(1, n):
      for k1 in range(1, n+1):
        for k2 in range(1, n+1):
          sol.add(Implies(Not(X[i][0][0]), Implies(And(X[i][j-1][k1], X[i][j][k2]), equals(D_par[i][j], D[k1-1][k2-1]))))     

        sol.add(Implies(Not(X[i][0][0]), Implies(And(X[i][j-1][k1], Not(at_least_one(X[i][j][1:]))), equals(D_par[i][j], D[k1-1][n]))))
      sol.add(Implies(Not(X[i][0][0]), Implies(And(X[i][j-1][0], X[i][j][0]), equals(D_par[i][j], D[n][n]))))

    # da  testare in un istanza dove un solo corriere porta tutto!!    
    for k in range(1, n+1):
      sol.add(Implies(X[i][-1][k], equals(D_par[i][-1], D[k-1][n])))

    sol.add(Implies(X[i][-1][0], equals(D_par[i][-1], D[n][n]))) # se il corriere non porta pacchi nell'istante j allora copia 0 nelle distanze
  
    # 2. compute the sum of the distances for each courier
  # for i in range(m):
    sol.add(sum_vec(D_par[i], D_tot[i], name= f"dist_{i}"))
  # 3. --NO save the maximum-- --> rho >= d1...dn
    # sol.add(lesseq(D_tot[i], rho))
  sol.add(maximum(D_tot,rho))

  # symmetry breaking constraints: only the first couriers (those with the most size) carry the most items
  for i in range(m-1):
    for j in range(n):
      sol.add(Implies(X[i][j][0], X[i+1][j][0]))

  sol.push()

  print("Time needed to encode the instance:", t.time() - start_time)

  UPPER_BOUND = maxDist
  LOWER_BOUND = np.max(instance.D[-1] + instance.D[:,-1])
  
  TIMEOUT = 300 * 1000
  sol.set('timeout', TIMEOUT)

  if sol.check() == unsat:
    return "Failed to solve"
  
  satisfiable = True
  
  previousModel = None
  
  start_time = t.time()


  
  while(satisfiable):
    MIDDLE_BOUND = (UPPER_BOUND + LOWER_BOUND) // 2
    middle_bits = toBinary(MIDDLE_BOUND, maxDistBin, BoolVal)  # notice the +0
    sol.add(lesseq(rho, middle_bits))
    print(f"search inside [{LOWER_BOUND}-{MIDDLE_BOUND}]: ", end= "")    
    
    current_time = t.time()
    past_time = int((current_time - start_time)*1000)
    sol.set('timeout', TIMEOUT - past_time)
    
    status = sol.check()
    
    print(status)
    if status == sat:
      model = sol.model()
      previousModel = model
      dist = [model.evaluate(rho[b]) for b in range(maxDistBin)]
      print(dist, toInteger(dist))
      UPPER_BOUND = toInteger(dist)

    elif status == unsat:
      sol.pop()
      sol.push()
      LOWER_BOUND = MIDDLE_BOUND
      
    elif status == unknown:
      print(f"The computation time exceeded the limit ({TIMEOUT // 1000} seconds)")      
      satisfiable = False
    
    if (UPPER_BOUND - LOWER_BOUND) == 0:
      satisfiable = False

  
  current_time = t.time()
  past_time = int((current_time - start_time)*1000)

  print("Time from beginning of the computation: ", past_time//1000)
  model = previousModel

  x = [[[ model.evaluate(X[i][j][k]) for k in range(0,n+1) ] for j in range(n) ] for i in range(m)]
  xD = [[[model.evaluate(D_par[i][j][b]) for b in range(maxDBin)] for j in range(n+1)] for i in range(m)]
  xDD = [[model.evaluate(D_tot[i][b]) for b in range(maxDistBin)] for i in range(m)]
  xO = [model.evaluate(rho[b]) for b in range(maxDistBin)]


  # print("X is")
  # # print(np.fromfunction(lambda i,j,k: 1 if x[i][j][k] == True else 0, shape= np.array(x).shape))
  # print(np.array(x))
  # print("D_par is")
  # print(np.array([[toInteger(np.array(xD[i][j])) for j in range(n+1)] for i in range(m)]))
  # print("D_tot is")
  # print(np.array(xDD))
  # # print("obj is")
  # # print(np.array(xO))

  # output  
  tot_s = []
  for i in range(m):
    sol = []
    for j in range(n):
      for k in range(0,n+1):
        if x[i][j][k] == True:
            sol.append(k)
    tot_s.append(sol)

  for i in range(m):
    print(f"Courier {i+1}:","deposit => ", end = "")
    for s in tot_s[i]:
      if s != 0 :
        print(s,"=> ", end = "")
    print("deposit")
  print("Distance travelled:")
  for i in range(m):
    print(f"Courier {i+1}: ", toInteger(np.array(xDD[i])))


  distances = [toInteger(np.array(xDD[i])) for i in range(m)]
  # print("Total distance: ", sum([toInteger(np.array(xDD[j]))for j in range(m)]))    
  # print(objHistory)
  print("Objective Function: ", max(distances))
  # print("Objective History: ", [toInteger(x) for x in objHistory])

  return model

instance = load_instance(16)

mod = mtps(instance)

print("")

Exception ignored in: <function AstRef.__del__ at 0x7f4d8c323c10>
Traceback (most recent call last):
  File "/home/matteo/.local/lib/python3.8/site-packages/z3/z3.py", line 352, in __del__
    Z3_dec_ref(self.ctx.ref(), self.as_ast())
  File "/home/matteo/.local/lib/python3.8/site-packages/z3/z3core.py", line 1577, in Z3_dec_ref
    _elems.f(a0, a1)
ctypes.ArgumentError: argument 1: <class 'KeyboardInterrupt'>: 


In [ ]:
instance.D

[[0, 3, 4, 5, 6, 6, 2],
 [3, 0, 1, 4, 5, 7, 3],
 [4, 1, 0, 5, 6, 6, 4],
 [4, 4, 5, 0, 3, 3, 2],
 [6, 7, 8, 3, 0, 2, 4],
 [6, 7, 8, 3, 2, 0, 4],
 [2, 3, 4, 3, 4, 4, 0]]